In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from  sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose

label_encoder = LabelEncoder()

InputLayer = keras.layers.InputLayer()
LSTM = keras.layers.LSTM
Dropout = keras.layers.Dropout
Dense = keras.layers.Dense

In [111]:
df = pd.read_csv('/Users/victoroliveira/Desktop/vonix-py-statistic/src/vonixstatisc/data/pandas_data/data_fluency_queue_april_after_24.csv')

df_to_dict = pd.DataFrame()
df = df.loc[df['queue_id'] == 'mgmsales']
df = df.loc[(df['handling_time'] > 10) & (df['handling_time'] < 200) ]
df = df.loc[df['agent_id'] == 1592]
df= df.dropna(axis=0)
df


,agent_id,queue_id,week_day,hour,minute,timestamp,handling_time
2,1592,mgmsales,1,21,6,1682370404,114
46,1592,mgmsales,1,21,26,1682371602,53
89,1592,mgmsales,1,21,36,1682372210,55
1580,1592,mgmsales,3,14,27,1682519242,129
2762,1592,mgmsales,3,20,44,1682541881,18
4241,1592,mgmsales,4,20,0,1682625625,56
4265,1592,mgmsales,4,20,5,1682625907,35
4373,1592,mgmsales,4,20,27,1682627273,74
4529,1592,mgmsales,4,21,3,1682629424,74
4846,1592,mgmsales,4,23,0,1682636449,77


In [112]:
df_to_dict['time'] = df['timestamp']
df_to_dict['ht'] = df['handling_time']

In [113]:
dict = df_to_dict.set_index('time').to_dict()['ht']
#dict

In [114]:
df

,agent_id,queue_id,week_day,hour,minute,timestamp,handling_time
2,1592,mgmsales,1,21,6,1682370404,114
46,1592,mgmsales,1,21,26,1682371602,53
89,1592,mgmsales,1,21,36,1682372210,55
1580,1592,mgmsales,3,14,27,1682519242,129
2762,1592,mgmsales,3,20,44,1682541881,18
4241,1592,mgmsales,4,20,0,1682625625,56
4265,1592,mgmsales,4,20,5,1682625907,35
4373,1592,mgmsales,4,20,27,1682627273,74
4529,1592,mgmsales,4,21,3,1682629424,74
4846,1592,mgmsales,4,23,0,1682636449,77


In [115]:
from datetime import datetime
from time import gmtime, strftime
import time
def transform_dict(data: dict, period: int) -> dict:
    """method that receives dict from database query
    {start_at(timestamp): talk_secs(seconds), ...} and create
    another dict based on the period (15, 30, 1hr) chosen
    {last_timestamp: [12,24,56,12, 90, 100], last_timestamp - period:[195,235,201,300], ... interval of three minutes for standard model}
    For standard model the data from the last period( 15 min, 30 min, 1h) is used to construct forecast for the next period so the return_dict begins with the last timestamp to the first
    """
    seconds = period * 60
    
    return_dict = {}

    keys_array = list(data.keys())
    beginning = keys_array[0]
    limit= beginning + seconds
    end = keys_array[-1]
    arr = []
    break_v = 1
    indice = 0
    while beginning < end:
        if break_v == beginning:
            break
        break_v = beginning
        for key, value in data.items():
            if key <= limit:
                arr.append(value)
                continue
        
            return_dict[f"{indice}"] = arr
            indice += 1
            arr = []
            arr.append(value)
            beginning = key
            limit = beginning + seconds
            

    return return_dict

In [116]:
dict_transformed = transform_dict(dict,15)

In [117]:
#dict_transformed

In [118]:
mean_object = {i: np.mean(dict_transformed[i]) for i in dict_transformed.keys()}

In [119]:
mean_object

{'0': 114.0,
 '1': 54.0,
 '2': 129.0,
 '3': 18.0,
 '4': 45.5,
 '5': 74.0,
 '6': 74.0,
 '7': 77.0,
 '8': 38.666666666666664,
 '9': 20.0,
 '10': 184.0,
 '11': 195.0,
 '12': 38.0,
 '13': 74.0,
 '14': 83.0,
 '15': 95.0,
 '16': 71.0,
 '17': 70.0}

In [120]:
cv_object = {i: np.std(dict_transformed[i]) for i in dict_transformed.keys()}

In [121]:
cv_object

{'0': 0.0,
 '1': 1.0,
 '2': 0.0,
 '3': 0.0,
 '4': 10.5,
 '5': 0.0,
 '6': 0.0,
 '7': 0.0,
 '8': 15.627610892974722,
 '9': 0.0,
 '10': 0.0,
 '11': 0.0,
 '12': 13.0,
 '13': 0.0,
 '14': 27.0,
 '15': 0.0,
 '16': 0.0,
 '17': 0.0}

In [122]:
call_volume = {i: len(dict_transformed[i]) for i in dict_transformed.keys()}

In [123]:
call_volume

{'0': 1,
 '1': 2,
 '2': 1,
 '3': 1,
 '4': 2,
 '5': 1,
 '6': 1,
 '7': 1,
 '8': 3,
 '9': 1,
 '10': 1,
 '11': 1,
 '12': 2,
 '13': 1,
 '14': 2,
 '15': 1,
 '16': 1,
 '17': 1}

In [124]:
df_to_ml = pd.DataFrame()
df_to_ml['call_volume'] = call_volume.values()
df_to_ml['cv'] = cv_object.values()
df_to_ml['mean'] = mean_object.values()
df_to_ml

,call_volume,cv,mean
0,1,0.000000,114.000000
1,2,1.000000,54.000000
2,1,0.000000,129.000000
3,1,0.000000,18.000000
4,2,10.500000,45.500000
5,1,0.000000,74.000000
6,1,0.000000,74.000000
7,1,0.000000,77.000000
8,3,15.627611,38.666667
9,1,0.000000,20.000000


In [125]:
np_to_save=df_to_ml.iloc[:, 0:3].values
np_to_save

array([[  1.        ,   0.        , 114.        ],
       [  2.        ,   1.        ,  54.        ],
       [  1.        ,   0.        , 129.        ],
       [  1.        ,   0.        ,  18.        ],
       [  2.        ,  10.5       ,  45.5       ],
       [  1.        ,   0.        ,  74.        ],
       [  1.        ,   0.        ,  74.        ],
       [  1.        ,   0.        ,  77.        ],
       [  3.        ,  15.62761089,  38.66666667],
       [  1.        ,   0.        ,  20.        ],
       [  1.        ,   0.        , 184.        ],
       [  1.        ,   0.        , 195.        ],
       [  2.        ,  13.        ,  38.        ],
       [  1.        ,   0.        ,  74.        ],
       [  2.        ,  27.        ,  83.        ],
       [  1.        ,   0.        ,  95.        ],
       [  1.        ,   0.        ,  71.        ],
       [  1.        ,   0.        ,  70.        ]])

In [126]:
np.savetxt("/Users/victoroliveira/Desktop/vonix-py-statistic/jupyter-studies/nn_models/nn_onequeue_allTimes/test_data.csv", np_to_save, delimiter=",")